# Notebook to add weather to parkrun data

In [ ]:
import requests
import pandas as pd
import time
import numpy as np
from functions.weather import fetch_weather

Start with the data produced from the initial notebook: 1_scrape_and_load.ipynb

In [ ]:
parkrun_df = pd.read_csv('data/raw/parkrun_full_data.csv')
parkrun_df['Date'] = pd.to_datetime(parkrun_df['Date'], format='%d/%m/%Y')

In [ ]:
# Define the latitude and longitude of the parkrun location
# If these are not supplied it defaults to the Brighton parkrun location

latitude=50.8425
longitude=-0.1718

Run the below cell to loop through all the dates in your parkrun data and get the weather data.

It will run the OpenMeteo API to get the temperature (°C), wind speed (km/h), and precipitation level (mm) for those dates.

In [ ]:
# Get the list of dates from the dataframe 'Date' column
date_list = list(pd.to_datetime(parkrun_df['Date'].unique()).strftime('%Y-%m-%d'))

# Initialize an empty list to store weather data
weather_data = []

# Fetch weather data for each date in the list
for i, date in enumerate(date_list):
    print(f"Fetching weather data for {date} ({i + 1}/{len(date_list)})...    ", end="\r")
    weather_data.append(fetch_weather(date, latitude, longitude))
    time.sleep(0.1)  # To avoid rate limits

# Convert the list of weather data into a DataFrame
weather_df = pd.DataFrame(weather_data)

In [ ]:
# Save the data to a CSV file for future use
weather_df.to_csv('data/raw/date_weather.csv', index=False)

The below cell merges the weather DataFrame with the initial parkrun DataFrame:

In [ ]:
weather_df = pd.read_csv('data/raw/date_weather.csv')                                 
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Merge the DataFrames on the 'Date' column
merged_df = parkrun_df.merge(weather_df, left_on='Date', right_on='date', how='left')

# Drop the duplicate 'date' column from weather_df
merged_df = merged_df.drop(columns=['date'])

# Display the first few rows of the merged DataFrame
merged_df.head()


In [ ]:
# Save the merged DataFrame for future use
merged_df.to_csv('data/raw/parkrun_weather.csv', index=False)